In [1]:
from pytezos import pytezos

In [2]:
# TODO: move this to settings.py
settings = dict(
    SHELL_URL = 'https://edonet-tezos.giganode.io/',
    # SHELL_URL = 'https://edonet.smartpy.io',
    # SHELL_URL = 'https://delphinet.smartpy.io',
    # SHELL_URL = 'https://delphinet-tezos.giganode.io/',

    # edonet
    CONTRACT_ADDRESS = 'KT1HbiznMedpC5BcQnUCUd5zCaXcemsU25Sk',

    # delphinet
    # CONTRACT_ADDRESS = 'KT18zjuSxBLdQqopnjoSeZWqqXkbEk9SnyDb',

    KEYS_DIRECTORY = 'test-keys',

    # default storage used to deploy new contract:
    DEFAULT_INITIAL_CONTRACT_STORAGE = {
        'betsAgainstLedger': 0,
        'betsAgainstSum': 0,
        'betsForLedger': 0,
        'betsForSum': 0,
        'closedRate': 0,
        'closedTime': 0,
        'currencyPair': 'XTZ-USD',
        'isBetsForWin': False,
        'isClosed': False,

        # edonet:
        'oracleAddress': 'KT1RCNpUEDjZAYhabjzgz1ZfxQijCDVMEaTZ',

        # delphinet:
        # 'oracleAddress': 'KT1Age13nBE2VXxTPjwVJiE8Jbt73kumwxYx',
        'targetRate': 0,
        'targetTime': 0
    },

    IS_ASYNC_ENABLED = False,
)

In [3]:
from os.path import join
from os import listdir
from pytezos import pytezos
from pprint import pprint


def load_key_filenames(directory):

    def make_key_name(fn):
        return fn.split('.')[0]

    filenames =  {
        make_key_name(fn): join(directory, fn)
        for fn in listdir(directory) if fn.lower().endswith('.json')
    }

    if not filenames:
        raise Exception(
            'Please add test keys into pytezos-jupyter-present/test-keys directory '
            + '(you can use https://faucet.tzalpha.net/')

    return filenames


In [4]:
from pytezos.operation.result import OperationResult


class KeysManager:
    def __init__(self, settings):

        self.settings = settings
        self.key_filenames = load_key_filenames(settings['KEYS_DIRECTORY'])
        self.shell_url = settings['SHELL_URL']
        self.is_async_enabled = settings['IS_ASYNC_ENABLED']

        self.pytezos_instances = {
            key_name: pytezos.using(key=key_filename, shell=self.shell_url)
            for key_name, key_filename in self.key_filenames.items()
        }

        assert len(self.pytezos_instances)
        print(f'Successfully loaded {len(self.pytezos_instances)} pytezos keys:')
        [print(f'- {key_name}') for key_name in self.pytezos_instances]


    def activate_keys(self):
        """ Runs activate_account for each loaded key """

        for key_name, pt in self.pytezos_instances.items():
            try:
                pt.activate_account().autofill().sign().inject(_async=self.is_async_enabled)
                pt.reveal().autofill().sign().inject(_async=self.is_async_enabled)
            except Exception as e:
                print(f'Error: {type(e)}, "{e}"')


class ContractManager:

    def __init__(self, pytezos, settings):
        self.settings = settings
        self.contract_address = settings['CONTRACT_ADDRESS']
        self.is_async_enabled = settings['IS_ASYNC_ENABLED']
        self.pytezos = pytezos
        self.contract = self.pytezos.contract(self.contract_address)


    def create_new_storage(self, **kwargs):
        """ Creates new storage for Fortune Crystal Ball smart contract with
            custom storage params in kwargs
        """

        storage = self.settings['DEFAULT_INITIAL_CONTRACT_STORAGE'].copy()
        storage.update(kwargs)
        return storage


    def find_originated_contract_address(self, new_contract_result):
        """ Searches for new originated contract address in blockchain """

        op_hash, branch = new_contract_result['hash'], new_contract_result['branch']
        print(f'hash: {op_hash}, branch: {branch}')

        blocks = cm.pytezos.shell.blocks[branch:]
        opg = blocks.find_operation(op_hash)
        res = OperationResult.from_operation_group(opg)
        originated_contract_address = res[0].originated_contracts[0]
        return originated_contract_address


    def deploy_new_contract(self, **kwargs):
        """ Deploys new contract with params transfered in kwargs """

        new_storage = self.create_new_storage(**kwargs)
        print(f'Deploying new contract with storage:')
        pprint(new_storage)

        new_contract = self.pytezos.origination(
            script=self.contract.script(initial_storage=new_storage))
        new_contract = new_contract.autofill().sign().inject(_async=self.is_async_enabled)

        originated_contract_address = self.find_originated_contract_address(new_contract)
        print(f'Contract successfully originated at address: {originated_contract_address}')

        # return new contract manager with replaced contract address:
        new_settings = self.settings.copy()
        new_settings.update(CONTRACT_ADDRESS=originated_contract_address)

        return ContractManager(self.pytezos, new_settings)

keys = KeysManager(settings)

# If your keys is not activated, you can activate it by running:
# keys.activate_keys()

Successfully loaded 5 pytezos keys:
- tz1RS9GoEXakf9iyBmSaheLMcakFRtzBXpWE
- tz1TdKuFwYgbPHHb7y1VvLH4xiwtAzcjwDjM
- tz1MdaJfWzP5pPx3gwPxfdLZTHW6js9havos
- tz1iQE8ijR5xVPffBUPFubwB9XQJuyD9qsoJ
- tz1ZAzDvkZCT2LAyPN8Kdxw3kes7xfWerZhZ


In [5]:
pt = keys.pytezos_instances['tz1TdKuFwYgbPHHb7y1VvLH4xiwtAzcjwDjM']

In [6]:
cm = ContractManager(keys.pytezos_instances['tz1TdKuFwYgbPHHb7y1VvLH4xiwtAzcjwDjM'], settings)

In [7]:
cm.contract.storage()

{'betsAgainstLedger': 14375,
 'betsAgainstSum': 414000,
 'betsForLedger': 14376,
 'betsForSum': 400000,
 'closedRate': 3621051,
 'closedTime': 1614474180,
 'currencyPair': 'XTZ-USD',
 'isBetsForWin': True,
 'isClosed': True,
 'oracleAddress': 'KT1RCNpUEDjZAYhabjzgz1ZfxQijCDVMEaTZ',
 'targetRate': 0,
 'targetTime': 0}

In [35]:
ncm = cm.deploy_new_contract(targetRate=4, targetTime=1)

Deploying new contract with storage:
{'betsAgainstLedger': 0,
 'betsAgainstSum': 0,
 'betsForLedger': 0,
 'betsForSum': 0,
 'closedRate': 0,
 'closedTime': 0,
 'currencyPair': 'XTZ-USD',
 'isBetsForWin': False,
 'isClosed': False,
 'oracleAddress': 'KT1RCNpUEDjZAYhabjzgz1ZfxQijCDVMEaTZ',
 'targetRate': 4,
 'targetTime': 1}
Wait 9 seconds until block BLF5Sf2Zy5umvFa2octo5bEpnRp1kH4gZ4gKWDVLoSgvDo2pJXo is finalized


2021-02-28 14:00:46.514 | DEBUG    | pytezos.rpc.search:find_operation:206 - checking level 46357...


hash: ontHwGmgzCEpJvnimJJQqVw1xGzZctjtDbu862n3vQoY7w9PeUS, branch: BKk7XoQpwp6S7JUwSfP8yFPP9SWRB5MFKcwfLz5ouvZPk4zmi4M


2021-02-28 14:00:47.483 | DEBUG    | pytezos.rpc.search:find_operation:206 - checking level 46306...
2021-02-28 14:00:47.647 | DEBUG    | pytezos.rpc.search:find_operation:206 - checking level 46307...
2021-02-28 14:00:47.846 | DEBUG    | pytezos.rpc.search:find_operation:206 - checking level 46308...
2021-02-28 14:00:48.009 | DEBUG    | pytezos.rpc.search:find_operation:206 - checking level 46309...
2021-02-28 14:00:48.243 | DEBUG    | pytezos.rpc.search:find_operation:206 - checking level 46310...
2021-02-28 14:00:48.494 | DEBUG    | pytezos.rpc.search:find_operation:206 - checking level 46311...
2021-02-28 14:00:48.639 | DEBUG    | pytezos.rpc.search:find_operation:206 - checking level 46312...
2021-02-28 14:00:48.869 | DEBUG    | pytezos.rpc.search:find_operation:206 - checking level 46313...
2021-02-28 14:00:49.056 | DEBUG    | pytezos.rpc.search:find_operation:206 - checking level 46314...
2021-02-28 14:00:49.252 | DEBUG    | pytezos.rpc.search:find_operation:206 - checking level

StopIteration: ontHwGmgzCEpJvnimJJQqVw1xGzZctjtDbu862n3vQoY7w9PeUS

In [9]:
new_settings = settings.copy()
new_settings['CONTRACT_ADDRESS'] = 'KT1AStyuCd5SCns6BtASqBcbs3LAudDJHfmF'

In [10]:
ncm = ContractManager(keys.pytezos_instances['tz1TdKuFwYgbPHHb7y1VvLH4xiwtAzcjwDjM'], new_settings)

In [38]:
# contract = cm.pytezos.contract('KT1V1x3nwbFU2WatcV6hBmCmCMdDB6E7BwLE')

In [ ]:
class CrystalContractManager(ContractManager):
    """ Python interface to work with Crystal Contract """

    def betAgainst(self, amount):
        self.contract.betAgainst().with_amount(414_000).as_transaction().autofill().sign().inject(_async=False)


### TODO:
+ сделать 5 ключей с помощью кранов
- сделать небольшой интерфейс для взаимодействия со смарт контрактом
    + выпуск нового смарт контракта на основе имеющегося
    - различные взамодействия с контрактом
    - написать пару тестов
- сделать красивую презентацию, с маркдауном, с графикой (распределение ставок например сделать в графиках)

In [9]:
# cm.contract.betAgainst().with_amount(414_000).as_transaction().autofill().sign().inject(_async=False)

In [14]:
cm.contract.betFor().with_amount(414_000).as_transaction().autofill().sign().inject(_async=False)

Wait 21 seconds until block BLofnqfrxHDrvmr49N8d5e3Q9GbkEL5ZJzMFMhjGXd8QGvGZyA6 is finalized


2021-02-28 13:49:02.075 | DEBUG    | pytezos.rpc.search:find_operation:206 - checking level 485178...


{'protocol': 'PsDELPH1Kxsxt8f9eWbxQeRxkjfbxoqM52jvs5Y5fBxWWh4ifpo',
 'chain_id': 'NetXm8tYqnMWky1',
 'hash': 'ooBfEE5M2r7wgeMGz8gXzwfCRXTZA8dr3ZTiav4HeJA7SAiDjLX',
 'branch': 'BM8w5z4XwNNezXcwzwo1hssRh4RdFcMidphZuD2yQh3tf8LHcjm',
 'contents': [{'kind': 'transaction',
   'source': 'tz1TdKuFwYgbPHHb7y1VvLH4xiwtAzcjwDjM',
   'fee': '1748',
   'counter': '1360210',
   'gas_limit': '14778',
   'storage_limit': '71',
   'amount': '414000',
   'destination': 'KT18zjuSxBLdQqopnjoSeZWqqXkbEk9SnyDb',
   'parameters': {'entrypoint': 'betFor', 'value': {'prim': 'Unit'}},
   'metadata': {'balance_updates': [{'kind': 'contract',
      'contract': 'tz1TdKuFwYgbPHHb7y1VvLH4xiwtAzcjwDjM',
      'change': '-1748'},
     {'kind': 'freezer',
      'category': 'fees',
      'delegate': 'tz1aWXP237BLwNHJcCD4b3DutCevhqq2T1Z9',
      'cycle': 236,
      'change': '1748'}],
    'operation_result': {'status': 'applied',
     'storage': {'prim': 'Pair',
      'args': [{'prim': 'Pair',
        'args': [{'prim': '

In [42]:
ncm.contract.storage()

{'betsAgainstLedger': 14517,
 'betsAgainstSum': 0,
 'betsForLedger': 14518,
 'betsForSum': 0,
 'closedRate': 0,
 'closedTime': 0,
 'currencyPair': 'XTZ-USD',
 'isBetsForWin': False,
 'isClosed': False,
 'oracleAddress': 'KT1RCNpUEDjZAYhabjzgz1ZfxQijCDVMEaTZ',
 'targetRate': 4,
 'targetTime': 1}

In [43]:
ncm.contract.close().as_transaction().autofill().sign().inject(_async=False)

Wait 26 seconds until block BMA82kb8mbn4c9FVa8X868FPkT4kmhPafuFmtcoTUCLNvQymuVS is finalized


2021-02-28 14:04:11.569 | DEBUG    | pytezos.rpc.search:find_operation:206 - checking level 46364...


{'protocol': 'PtEdo2ZkT9oKpimTah6x2embF25oss54njMuPzkJTEi5RqfdZFA',
 'chain_id': 'NetXSgo1ZT2DRUG',
 'hash': 'onkCrjtJNJEQchQgiQgN1d7d4Mgpxgo7m9kahKNAq5isiEWLkvd',
 'branch': 'BLaLPBYCai9bgvho8wJ3MAkh4JW89VEDYQwdd1UzcBUiSMFguG9',
 'contents': [{'kind': 'transaction',
   'source': 'tz1TdKuFwYgbPHHb7y1VvLH4xiwtAzcjwDjM',
   'fee': '9696',
   'counter': '93023',
   'gas_limit': '94282',
   'storage_limit': '7',
   'amount': '0',
   'destination': 'KT1AStyuCd5SCns6BtASqBcbs3LAudDJHfmF',
   'parameters': {'entrypoint': 'close', 'value': {'prim': 'Unit'}},
   'metadata': {'balance_updates': [{'kind': 'contract',
      'contract': 'tz1TdKuFwYgbPHHb7y1VvLH4xiwtAzcjwDjM',
      'change': '-9696'},
     {'kind': 'freezer',
      'category': 'fees',
      'delegate': 'tz1aWXP237BLwNHJcCD4b3DutCevhqq2T1Z9',
      'cycle': 22,
      'change': '9696'}],
    'operation_result': {'status': 'applied',
     'storage': [[{'prim': 'Pair',
        'args': [{'prim': 'Pair', 'args': [{'int': '14517'}, {'int'

In [ ]:
crystal_ball.withdraw().as_transaction().autofill().sign().inject(_async=False)